In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=52cf76a60d45bc9199d669e223583c58ebdd7a5f02594ade50ce6d26a308e383
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, broadcast, when
from pyspark.sql.types import FloatType, StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import matplotlib.pyplot as plt

# Initialize Spark Session
spark = SparkSession.builder.master("local[*]").appName("ClassificationModels").getOrCreate()

# Load the dataset
data = spark.read.csv('/content/drive/MyDrive/part-00000-088a67a0-1eba-42ac-af32-e6ff21d882e4-c000.csv', inferSchema=True, header=True)
data = data.dropna(subset=["user_id", "text", "stars"]).drop('date', 'useful', 'funny', 'cool')
data = data.withColumn("stars", col("stars").cast("double"))

# Regex to match user ID pattern
user_id_pattern = r'^[a-zA-Z0-9_-]{22}$'
data = data.filter(col('user_id').rlike(user_id_pattern))

# Limit the DataFrame to 1 million rows
data = data.limit(1000000)

# Calculate average stars and categorize reviewers
user_avg_ratings = data.groupBy("user_id").agg(avg("stars").alias("avg_stars"))
data = data.join(broadcast(user_avg_ratings), "user_id")
data = data.withColumn("category", when(col("avg_stars") >= 4, "generous").otherwise(when(col("avg_stars") <= 2, "picky").otherwise("normal")))

# NLP and Feature Extraction
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
assembler = VectorAssembler(inputCols=["features"], outputCol="features_vector")
labelIndexer = StringIndexer(inputCol="category", outputCol="label")

# Configure the pipeline for data transformation
pipeline_stages = [tokenizer, remover, hashingTF, idf, assembler, labelIndexer]
pipeline = Pipeline(stages=pipeline_stages)
prepared_data = pipeline.fit(data).transform(data)

# Split data into training and testing sets
train_data, test_data = prepared_data.randomSplit([0.7, 0.3], seed=42)

# Define classifiers
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, featuresCol="features_vector", labelCol="label")
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol="features_vector", labelCol="label")

# Fit models on training data
lr_model = lr.fit(train_data)
nb_model = nb.fit(train_data)

# Evaluate models on test data
lr_predictions = lr_model.transform(test_data)
nb_predictions = nb_model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lr_predictions)
nb_accuracy = evaluator.evaluate(nb_predictions)

evaluator.setMetricName("f1")
lr_f1 = evaluator.evaluate(lr_predictions)
nb_f1 = evaluator.evaluate(nb_predictions)

# Visualization of model performance
models = ['Logistic Regression', 'Naive Bayes']
accuracies = [lr_accuracy, nb_accuracy]
f1_scores = [lr_f1, nb_f1]
fig, ax = plt.subplots(1, 2, figsize=(14, 5))
ax[0].bar(models, accuracies, color=['blue', 'green'])
ax[0].set_title('Model Accuracy Comparison')
ax[0].set_ylabel('Accuracy')
ax[1].bar(models, f1_scores, color=['blue', 'green'])
ax[1].set_title('Model F1-Score Comparison')
plt.show()
